In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [6]:
file_url = 'data/data.csv'
dataframe = pd.read_csv(file_url)
print(dataframe.shape)
dataframe.head(20)

(500, 10)


,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date
0,1,Elevator,7,103,1/6/2020,Manufacturer_4,39313,0,1,6/19/2023
1,2,Plumbing System,6,109,6/11/2022,Manufacturer_2,20012,0,1,1/6/2023
2,3,Fire Alarm,3,104,7/24/2019,Manufacturer_4,21546,10,5,9/24/2022
3,4,Elevator,1,105,1/4/2021,Manufacturer_4,47038,1,1,5/23/2023
4,5,Plumbing System,2,104,11/28/2022,Manufacturer_1,20022,10,3,1/29/2023
5,6,Fire Alarm,2,103,7/29/2021,Manufacturer_3,1602,9,4,10/3/2022
6,7,Fire Alarm,6,108,8/23/2020,Manufacturer_2,15650,2,3,1/13/2023
7,8,HVAC,2,106,6/3/2021,Manufacturer_2,33932,1,4,8/9/2023
8,9,Fire Alarm,6,108,7/30/2023,Manufacturer_5,24332,5,4,2/13/2023
9,10,Elevator,7,101,7/24/2019,Manufacturer_4,18722,10,2,5/29/2023


In [4]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 400 samples for training and 100 for validation
